In [19]:
from datetime import datetime
import json
import math
import pandas as pd
import requests

In [20]:
url = "https://bgapidatafeed.vps.com.vn/listvn30"
response = requests.get(url)
vn30 = response.json()
vn30_file = open("./vn30.json", "w+")
vn30_json = json.dumps(vn30, indent=2)
vn30_file.write(vn30_json)

272

In [21]:
url = "https://bgapidatafeed.vps.com.vn/getlistallstock"
response = requests.get(url)
symbols : list[dict] = response.json()
symbols_data_frame = pd.DataFrame(symbols)
symbols_data_frame.to_csv("./symbols.csv", index=False)

In [22]:
timestamp = math.floor(datetime.now().timestamp())
timestamp

1700138433

In [23]:
def get_history(symbol : str):
    now = math.floor(datetime.now().timestamp())
    url = f"https://histdatafeed.vps.com.vn/tradingview/history?symbol={symbol}&resolution=1D&from=0&to={now}"
    response = requests.get(url)
    history_data : dict = response.json()
    symbol = history_data.get("symbol", "")
    t : list[int] = history_data.get("t", [])
    o : list[float] = history_data.get("o", [])
    h : list[float] = history_data.get("h", [])
    l : list[float] = history_data.get("l", [])
    c : list[float] = history_data.get("c", [])
    v : list[int] = history_data.get("v", [])
    history : list[dict] = []
    if len(t) > 0:
        for index, timestamp in enumerate(t):
            date = datetime.fromtimestamp(timestamp).date()
            history.append({
                "date": date,
                "timestamp": timestamp,
                "open": o[index],
                "high": h[index],
                "low": l[index],
                "close": c[index],
                "volume": v[index]
            })
        history_data_frame = pd.DataFrame(history)
        history_data_frame.to_csv(f"./history/{symbol}.csv", index=False)
    

In [24]:
for symbol in vn30:
    get_history(symbol)